**Import bibliotek + import bazy danych**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from numpy import where
from numpy import meshgrid
from numpy import arange
from numpy import hstack

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

diabetes = pd.read_csv('diabetes.csv')

features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
target = 'Diabetic'
X, y = diabetes[features], diabetes[target]

**Podział danych na zbiór treningowy oraz zbiór testowy + standaryzacja zbiorów**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X[['Pregnancies', 'Age']], y, test_size=0.30, random_state=0, stratify=y)

scaler_2var = StandardScaler()
X_train_standardized = scaler_2var.fit_transform(X_train)
X_test_standardized = scaler_2var.transform(X_test)

**Zdefiniowanie funkcji: plot_classification_surface:**

In [ ]:
def generate_model_LR(penalty,C):
    global model_list
    
    if penalty=="l1":
        model = LogisticRegression(penalty=penalty,C=C,solver='liblinear')
    elif penalty=="l2":
        model = LogisticRegression(penalty=penalty,C=C)
    elif penalty=="elasticnet":
        model = LogisticRegression(penalty=penalty,C=C,solver='saga', l1_ratio=0.1)
    else:
        raise ValueError("Nieprawidłowa wartość 'penalty'")
    
    return model

**Zdefiniowanie funkcji: plot_classification_surface:**

In [ ]:
def plot_classification_surface(X_plot, y_plot, trained_model):
    plt.figure(figsize=(12, 7))

    min1, max1 = X_plot[:, 0].min()-1, X_plot[:, 0].max()+1
    min2, max2 = X_plot[:, 1].min()-1, X_plot[:, 1].max()+1

    x1grid = arange(min1, max1, 0.1)
    x2grid = arange(min2, max2, 0.1)

    xx, yy = meshgrid(x1grid, x2grid)

    r1, r2 = xx.flatten(), yy.flatten()
    r1, r2 = r1.reshape((len(r1), 1)), r2.reshape((len(r2), 1))

    grid = hstack((r1,r2))

    yhat = trained_model.predict(grid)

    zz = yhat.reshape(xx.shape)

    plt.contourf(xx, yy, zz, cmap='Paired')

    for class_value in range(2):

        row_ix = where(y_plot == class_value)
        plt.scatter(X_plot[row_ix, 0], X_plot[row_ix, 1], cmap='Paired', alpha=0.3, label=class_value)

    plt.legend(loc='upper right')
    plt.show()

**Zdefiniowanie funkcji calculate_metrics_new:**

In [ ]:
metrics_dataframe = pd.DataFrame(columns=[
    'Model', 'Penalty', 'Par_C','F1_train',
    'F1_test', 'AUC', 'Accuracy', 'Precision', 'Recall'
])

def calculate_metrics_new(model, name, X_test, y_test, X_train, y_train):
    global metrics_dataframe
    predictions_test = model.predict(X_test)
    predictions_train = model.predict(X_train)
    predictions_proba = model.predict_proba(X_test)[:, 1]

    try:
        name_title = f"{name}"
        penalty = model.get_params()['penalty']
        C_value = model.get_params()['C']
    except:
        penalty = "UNKNOWN"
        C_value = "UNKNOWN"    
    
    print(f"\n Model: {name_title} - penalty:{penalty}, parametr C:{C_value}")
   
    plot_classification_surface(X_train, y_train, model)
    
    f1_metric_train = round(f1_score(y_train, predictions_train),4)
    f1_metric_test = round(f1_score(y_test, predictions_test),4)
    auc_metric = round(roc_auc_score(y_test, predictions_proba),4)
    acc_metric = round(accuracy_score(y_test, predictions_test),4)
    prec_metric = round(precision_score(y_test, predictions_test),4)
    rec_metric = round(recall_score(y_test, predictions_test),4)

    new_row = pd.DataFrame([{
        'Model': name,
        'Penalty': penalty,
        'Par_C': C_value,
        'F1_train': f1_metric_train,
        'F1_test': f1_metric_test,
        'AUC': auc_metric,
        'Accuracy': acc_metric,
        'Precision': prec_metric,
        'Recall': rec_metric
    }])

    metrics_dataframe = pd.concat([metrics_dataframe, new_row], ignore_index=True)
    return metrics_dataframe

**Zdefiniowanie zbiorów parametrów: "penalties" i "parametrs_C"**

In [ ]:
penalties = ['l1', 'l2', 'elasticnet']
parametrs_C = [0.01, 0.1, 1, 10, 100]

**Uruchomienie pętli generującej, trenującej modele oraz wyświetlającej zbiór metryk wygenerownych modeli**

In [ ]:
for penalty in penalties:
    for C in parametrs_C:
      label = f"Logistic Regression"  
      model = generate_model_LR(penalty,C)
      model.fit(X_train_standardized,y_train)
      metrics_dataframe = calculate_metrics_new(model, label, X_test_standardized, y_test, X_train_standardized, y_train)
display(metrics_dataframe)

**WNIOSKI:**

In [ ]:
print(f"Modele regresji logistycznej z różnymi rodzajami regularyzacji: {penalties} oraz ich stopniem: {parametrs_C} wskazują praktycznie takie same wyniki dla metryk takich jak:")
print(f"Dokładność (Accuracy) – zakres: {round(metrics_dataframe['Accuracy'].min(), 4)} - {round(metrics_dataframe['Accuracy'].max(), 4)}")
print(f"Precyzja (Precision) – zakres: {round(metrics_dataframe['Precision'].min(), 4)} - {round(metrics_dataframe['Precision'].max(), 4)}")
print(f"Czułość (Recall) – zakres: {round(metrics_dataframe['Recall'].min(), 4)} - {round(metrics_dataframe['Recall'].max(), 4)}")

print("\nModel regresji logistycznej nie uzyskuje dobrych wyników – nawet przy zmianie wartości regularyzacji i jego stopnia, metryki pozostają dość niskie.")
print("Oznacza to, że na podstawie jedynie zmiennych 'Pregnancies' oraz 'Age' model nie jest w stanie skutecznie przewidywać zmiennej 'Diabetics'.")

print(f"\nMetrykę F1:")
print(f"Dla zbiorów treningowych – zakres F1: {round(metrics_dataframe['F1_train'].min(), 4)} - {round(metrics_dataframe['F1_train'].max(), 4)}")
print(f"Dla zbiorów testowych – zakres F1: {round(metrics_dataframe['F1_test'].min(), 4)} - {round(metrics_dataframe['F1_test'].max(), 4)}")

print("\nWeryfikując metrykę F1 nie widać oznak przetrenowania – dla danych testowych F1 nie różni się znacznie od F1 dla danych treningowych.")
print("\nModel raczej wymaga dotrenowania, biorąc pod uwagę ogólnie niski poziom F1 dla obu zbiorów danych.")
